In [ ]:
import scipy.stats as stats
import numpy as np
import scipy.optimize as optimize
import math
import matplotlib.pyplot as plt
import pandas as pd
import torch
import itertools
from itertools import combinations_with_replacement
from itertools import combinations
from itertools import permutations

import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 200

log_plot = True

import bo_methods_lib

#Note: Need to normalize all values

In [ ]:
def grid_sampling(num_points, bounds):
        """
        Generates Grid sampled data
        
        Parameters
        ----------
        num_points: int, number of points in LHS, should be greater than # of dimensions
        bounds: ndarray, array containing upper and lower bounds of elements in LHS sample. Defaults of 0 and 1
        
        Returns:
        ----------
        grid_data: ndarray, (num_points)**bounds.shape[1] grid sample of data
        
        """
        #Generate mesh_grid data for theta_set in 2D
        #Define linspace for theta
        params = np.linspace(0,1, num_points)
        #Define dimensions of parameter
        dimensions = bounds.shape[1]
        #Generate the equivalent of all meshgrid points
        df = pd.DataFrame(list(itertools.product(params, repeat=dimensions)))
        df2 = df.drop_duplicates()
        scaled_data = df2.to_numpy()
        #Normalize to bounds 
        lower_bound = bounds[0]
        upper_bound = bounds[1]
        grid_data = scaled_data*(upper_bound - lower_bound) + lower_bound 
        return grid_data
    
def calc_muller(model_coefficients, x):
    """
    Caclulates the Muller Potential
    
    Parameters
    ----------
        model_coefficients: ndarray, The array containing the values of Muller constants
        x: ndarray, Values of X
        noise: ndarray, Any noise associated with the model calculation
    
    Returns:
    --------
        y_mul: float, value of Muller potential
    """
    #Reshape x to matrix form
    #If array is not 2D, give it shape (len(array), 1)
    if not len(x.shape) > 1:
        x = x.reshape(-1,1)
        
    assert x.shape[0] == 2, "Muller Potential x_data must be 2 dimensional"
    X1, X2 = x #Split x into 2 parts by splitting the rows
    
    #Separate all model parameters into their appropriate pieces
    model_coefficients_reshape = model_coefficients.reshape(6, 4)
        
    #Calculate Muller Potential
    A, a, b, c, x0, y0 = model_coefficients_reshape
    term1 = a*(X1 - x0)**2
    term2 = b*(X1 - x0)*(X2 - y0)
    term3 = c*(X2 - y0)**2
    y_mul = np.sum(A*np.exp(term1 + term2 + term3) )
    
    return y_mul

In [ ]:
%%time
CS = 2.2
skip_params = 0

#Change this to get less parameters
# Create synthetic data assuming the following values for theta
# a_guess = np.array([-150,-75,-150,10, #A guess
#                     0.5,0.5,-7,1, #a guess
#                     -1, 1, 10, 1,#b guess
#                     -8, -8, -8, 0,#c guess
#                     0.5, 0.5, 0, 0,#x0 guess
#                     0.5, 0.5, 0.5, 0.5]) #y0 guess

# a_guess = np.array([-150,-75,-150,10, #A guess
#                     0.5,0.5,-7,1, #a guess
#                     -1, 1, 10, 1,#b guess
#                     -8, -8, -8, 0,#c guess
#                     0.5, 0.5, 0, 0]) #x0 guess

a_guess = np.array([-100, -90, -100, 5]) #A guess

# a_guess = np.array([-1, 1, 10, 1]) #b guess

Constants = np.array([[-200,-100,-170,15],
                      [-1,-1,-6.5,0.7],
                      [0,0,11,0.6],
                      [-10,-10,-6.5,0.7],
                      [1,0,-0.5,-1],
                      [0,0.5,1.5,1]])

num_param_guess = int(len(a_guess)/Constants.shape[1])
##New Cell
# Evaluate model and add noise based on assumed theta values
# This generates experimental data points
num_points = 5
bounds_x = np.array([[-1.5, -0.5], 
                     [1, 2]])
Xexp = grid_sampling(num_points, bounds_x)
Yexp = [calc_muller(Constants, Xexp[i]) for i in range(len(Xexp)) ]
print(Xexp.shape)
print(a_guess[0])
print(Constants.shape)

In [ ]:
##New Cell

# Evaluate model based on the assumed experimental values
#Create Meshgrid for X1 and X2 and evaluate Y
len_mesh_data = 20
x1 = np.linspace(np.min(Xexp[:,0]),np.max(Xexp[:,0]),len_mesh_data)
x2 = np.linspace(np.min(Xexp[:,1]),np.max(Xexp[:,1]),len_mesh_data)
X1, X2 = np.meshgrid(x1,x2)
X_mesh = np.meshgrid(x1,x2)
# #Creates an array for Y that will be filled with the for loop
# #Initialize y_sim
Y = [] #len_data 

# #Find evey combination of X1/X2 to find the SSE for each combination
#Set constants
A, a, b, c, x0, y0 = Constants

#Calculate y_sim
#Define X1 and X2 (Need a better way do do this without for loops)
#Loop over combinations of X1 X2
for i in range(len_mesh_data):
    for j in range(len_mesh_data):
        Term1 = a*(X1[i,j] - x0)**2
        Term2 = b*(X1[i,j] - x0)*(X2[i,j] - y0)
        Term3 = c*(X2[i,j] - y0)**2
        Y.append( np.sum( A*np.exp(Term1 + Term2 + Term3) ) )
        
#Reshape to correct dimension (Is there an easier wat to do this?)        
Y = np.array(Y).reshape(len_mesh_data,-1)
# print(Y)
# Compare the experiments to the true model
fig = plt.figure()
ax = plt.axes(projection='3d')
ax.contour3D(X1, X2, Y, 100, cmap='Reds')
ax.plot(1000,1000,1000, label = "$y_{exp}$", color = 'red')
ax.scatter(1000,1000,1000, label = "Exp Data", color = 'green', edgecolors = "k")
ax.scatter3D(Xexp[:,0], Xexp[:,1], Yexp, c=Yexp, cmap='Greens', edgecolors = "k")
plt.legend(fontsize=10,bbox_to_anchor=(0, 1.0, 1, 0.2),borderaxespad=0, loc = "lower right")

ax.minorticks_on() # turn on minor ticks
ax.tick_params(direction="in",top=True, right=True) 
ax.tick_params(which="minor",direction="in",top=True, right=True)


ax.zaxis.set_tick_params(labelsize=12)
ax.yaxis.set_tick_params(labelsize=12)
ax.xaxis.set_tick_params(labelsize=12)
ax.grid(False)

ax.set_xlim((np.amin(X1),np.amax(X1)))
ax.set_ylim((np.amin(X2),np.amax(X2)))

ax.set_xlabel('X1', fontsize=16,fontweight='bold')
ax.set_ylabel('X2', fontsize=16,fontweight='bold')
ax.set_zlabel('Muller Potential',fontsize=16,fontweight='bold');

ax.locator_params(axis='y', nbins=5)
ax.locator_params(axis='x', nbins=5)
# ax.locator_params(axis='z', nbins=5)
# plt.title("Plotting True Model and Synthetic Data")
plt.show()

# Y_scaled = np.array(Y_scaled).reshape(len_mesh_data,-1)
# Y_bounds = np.array([np.min(Y_scaled), np.max(Y_scaled)])
# Y = values_scaled_to_real(Y_scaled, Y_bounds)
# Y = Y.reshape(len_mesh_data,-1)

In [ ]:
##New Cell

## define function that includes nonlinear model
def model(a_guess, Constants, x, skip_params = 0):
    '''
        """
    Creates Muller potential values given a guess for "a"
    Parameters
    ----------
        a_guess: ndarray, guess value for a
        Constants: ndarray, The array containing the true values of Muller constants
        x: ndarray, Independent variable data (exp or pred)
    Returns
    -------
        y_model: ndarray, The simulated Muller potential given the guess
    '''
    #Assert statements check that the types defined in the doctring are satisfied
    
    #Converts parameters to numpy arrays if they are tensors
    if torch.is_tensor(a_guess)==True:
        a_guess = a_guess.numpy()
        
    if isinstance(a_guess, pd.DataFrame):
        a_guess = a_guess.to_numpy()
    
    #Initialize y_sim, set len_data and dim_x
    len_x_shape = len(x.shape) #Will tell us whether we're looking at Xexp or Xmesh
        
    if len_x_shape < 3:
        len_x_data = x.shape[0]
        y_model = np.zeros(len_x_data)
        
    else:
        len_x_data = x.shape[1]
        y_model = np.zeros((len_x_data, len_x_data))
    
    #Set dig out values of a from train_p
    #Set constants to change the a row to the index of the first loop
    Constants_local = np.copy(Constants)
    dim_guess = int(len(a_guess)/Constants.shape[1])
    a_guess = a_guess.reshape(dim_guess,-1)
    
    for i in range(dim_guess):
        Constants_local[i+skip_params] = a_guess[i]#Since we've chosen A, a, b, c, x0, and y0
        
#     print(Constants_local)
#     print(Constants_local)
    A, a, b, c, x0, y0 = Constants_local
#     print(a,A)

    #Iterates over Xexp to find the y for each combination
    for i in range(len_x_data):
        #Calculate y_sim
        if len_x_shape < 3:
            X1, X2 = x[i,0], x[i,1]
            Term1 = a*(X1 - x0)**2
            Term2 = b*(X1 - x0)*(X2 - y0)
            Term3 = c*(X2 - y0)**2
            y_model[i] = np.sum(A*np.exp(Term1 + Term2 + Term3) )
        else:
        #loop over all i and j
            X1, X2 = x
            for i in range(len_x_data):
                for j in range(len_x_data):
                    Term1 = a*(X1[i,j] - x0)**2
                    Term2 = b*(X1[i,j] - x0)*(X2[i,j] - y0)
                    Term3 = c*(X2[i,j] - y0)**2
                    y_model[i,j] = ( np.sum( A*np.exp(Term1 + Term2 + Term3) ) )
   
    if not len_x_shape < 3:
        y_model = y_model.reshape(len_x_data, -1)
    
    return y_model

# print(model(a_guess,Constants,Xexp, skip_params))

##New Cell

# Create a function to optimize, in this case, least squares fitting
def regression_func(a_guess, Constants, x, y, skip_params = 0):
    '''
    Function to define regression function for least-squares fitting
    Arguments:
        a_guess: ndarray, guess value for a
        Constants: ndarray, The array containing the true values of Muller constants
        x: ndarray, experimental X data (Inependent Variable)
        y: ndarray, experimental Y data (Dependent Variable)
    Returns:
        e: residual vector
    '''
    
    error = y - model(a_guess, Constants, x, skip_params); #NOTE: Least squares will calculate sse based off this to minimize
    
    return error

# print(regression_func(a_guess, Constants, Xexp, Yexp))


In [ ]:
%%time
## specify initial guess
a0 = a_guess
#Define # of dimensions
try:
    d = a_guess.shape[0]*a_guess.shape[1]
except:
    d = a_guess.shape[0]
## specify bounds
# first array: lower bounds
# second array: upper bounds
lower = np.repeat(-np.inf, d)
upper = np.repeat(np.inf, d)
bounds = (lower, upper) 

## use least squares optimizer in scipy
# argument 1: function that takes theta as input, returns residual
# argument 2: initial guess for theta
# optional arguments 'bounds': bounds for theta
# optional arugment 'args': additional arguments to pass to residual function
# optional argument 'method': select the numerical method
#   if you want to consider bounds, choose 'trf'
#   if you do not want to consider bounds, try either 'lm' or 'trf'
Solution = optimize.least_squares(regression_func, a0 ,bounds=bounds, method='trf',args=(Constants, Xexp, Yexp, skip_params),verbose=2)

a_model = Solution.x
a_model_soln = a_model.reshape(num_param_guess,-1)
print("a = ",a_model_soln)
print("Constants", Constants)

In [ ]:
# a1, a2, a3, a4 = np.meshgrid(a1_lin, a2_lin, a3_lin, a4_lin)
# a_guesses = np.meshgrid(a1_lin, a2_lin, a3_lin, a4_lin)

# P_inds = np.array([0,3])
# print(sse_func(a_model, Xexp, Yexp, P_inds, a1_lin, a2_lin))

In [ ]:
#New Cell
X_pred = np.array(np.meshgrid(x1,x2))
Y_pred = model(a_model, Constants, X_pred, skip_params)


# create plot and compare predictions and experiments
fig = plt.figure(figsize = (6.4,4))
ax = plt.axes(projection='3d')
ax.contour3D(X1, X2, Y_pred, 100, cmap='Blues')
ax.contour3D(X1, X2, Y, 100, cmap='Reds')
ax.scatter3D(Xexp[:,0], Xexp[:,1], Yexp, c=Yexp, cmap='Greens', edgecolors = "k")
ax.plot(1000,1000,1000, label = "$y_{sim}$", color = 'blue')
ax.plot(1000,1000,1000, label = "$y_{exp}$", color = 'red')
ax.scatter(1000,1000,1000, label = "Exp Data", color = 'green', edgecolors = "k")
plt.legend(fontsize=10,bbox_to_anchor=(0, 1.0, 1, 0.2),borderaxespad=0, loc = "lower right")

ax.minorticks_on() # turn on minor ticks
ax.tick_params(direction="in",top=True, right=True) 
ax.tick_params(which="minor",direction="in",top=True, right=True)

ax.locator_params(axis='y', nbins=5)
ax.locator_params(axis='x', nbins=5)
ax.locator_params(axis='z', nbins=5)

ax.zaxis.set_tick_params(labelsize=12)
ax.yaxis.set_tick_params(labelsize=12)
ax.xaxis.set_tick_params(labelsize=12)
ax.grid(False)

ax.set_xlim((np.amin(X1),np.amax(X1)))
ax.set_zlim(np.amin(Y),np.amax(Y))
ax.set_xlim(np.amin(X1),np.amax(X1))
ax.set_ylim(np.amin(X2),np.amax(X2))
ax.set_xlabel('X1', fontsize=16,fontweight='bold')
ax.set_ylabel('X2', fontsize=16,fontweight='bold')
ax.set_zlabel('Muller Potential', fontsize=16,fontweight='bold')
ax.zaxis._axinfo['label']['space_factor'] = 2
# plt.title("Predictions vs Synthetic Data")
plt.show()

# plt.savefig("Figures/sim_true_comp.png",dpi=300)
# plt.show()

In [ ]:
#Plot error
Y_pred_of_exp = model(a_model, Constants, Xexp, skip_params)
error = (Yexp - Y_pred_of_exp)
print("SSE = ", np.sum(error**2))
plt.plot(Y_pred_of_exp,error,"b.",markersize=20, label = "Error")

plt.legend(fontsize=10,bbox_to_anchor=(0, 1.05, 1, 0.2),borderaxespad=0, loc = "lower right")

plt.minorticks_on() # turn on minor ticks
plt.tick_params(direction="in",top=True, right=True) 
plt.tick_params(which="minor",direction="in",top=True, right=True)

plt.locator_params(axis='y', nbins=5)
plt.locator_params(axis='x', nbins=5)

plt.xticks(fontsize=16)
plt.yticks(fontsize=16)

plt.grid(False)

# plt.title("Residuals")
plt.xlabel('Predicted Potential', fontsize=16,fontweight='bold')
plt.ylabel('Residuals', fontsize=16,fontweight='bold')
plt.show()

In [ ]:
# def sse_func(a_model, x, y, P_inds, P1_vals, P2_vals, skip_params):
def sse_func(a_model, x, y, meshgrid, P_inds, skip_params):
    '''
    Function to define define sum of squared error function for heat map
    Arguments:
        xx: An N X D array of all a_1 values
        yy: An D X N array of all a_2 values
        x: independent variable vector (predicted x values including noise)
        y: dependent variable vector (predicted y values on Heat Map)
    Returns:
        sse: N x N sum of squared error matrix of all generated combination of xx and yy
    '''
#     #Meshgrid
#     P1_mesh, P2_mesh = np.meshgrid(P1_vals,P2_vals)
    xx, yy = meshgrid
    #Copy Center Point
    a_guess_local = np.copy(a_model)
    #Initialize SSE Maxtrix
    sse = np.zeros((xx.shape))
#     print(P_inds)
#     print(a_guess_local)
    #Calculate SSE
    for i in range(len(xx)):
        for j in range(len(yy)):
#             a_guess_local[P_inds[0]+ 4*skip_params] = xx[i,j]
#             a_guess_local[P_inds[1]+ 4*skip_params] = yy[i,j]
            a_guess_local[P_inds[0] - 4*skip_params] = xx[i,j]
            a_guess_local[P_inds[1] - 4*skip_params] = yy[i,j]
#             print(a_guess_local)
            sse[i,j] = sum((y - model(a_guess_local, Constants, x, skip_params))**2) 
    
    return sse

In [ ]:
#New Cell
# log_plot = False
log_plot = True
# save_figure = True
save_figure = False

# generate predictions
X_pred = np.array(X_mesh)
Y_pred = model(a_guess, Constants, X_pred, skip_params)
Constants_2 = Constants.flatten()

#Modify these to have less parameters
#Generate Guesses for a1-a4
a1_lin = a2_lin = np.linspace(-2,0,10)
a3_lin = np.linspace(-10,2,10)
a4_lin = b1_lin = b2_lin = b4_lin = c4_lin = np.linspace(-2,2,10)
A1_lin = A2_lin = A3_lin = np.linspace(-250,50,10)
A4_lin = np.linspace(10,20,10)
b3_lin = np.linspace(8,12,10)
b4_lin = np.linspace(-2,2,10)
c1_lin = np.linspace(-12,-8,10)
c2_lin = np.linspace(-12,-8,10)
c3_lin = np.linspace(-10,-6,10)

x0_1_lin = x0_2_lin = x0_3_lin = x0_4_lin = np.linspace(-2,2,10)
y0_1_lin = y0_2_lin = y0_3_lin = y0_4_lin = np.linspace(-2,2,10)

# param_dict = {0: "A1", 1: "A2", 2: "A3", 3: "A4",
#               4: "a1", 5: "a2", 6: "a3", 7: "a4",
#               8: "b1", 9: "b2", 10:"b3", 11:"b4",
#               12:"c1", 13:"c2", 14:"c3", 15:"c4",
#               16:"x0_1", 17:"x0_2", 18:"x0_3", 19:"x0_4",
#               20:"y0_1", 21:"y0_2", 22:"y0_3", 23:"y0_4"}

# param_dict_vals = {0: A1_lin, 1: A2_lin, 2: A3_lin, 3: A4_lin,
#               4: a1_lin, 5: a2_lin, 6: a3_lin, 7: a4_lin,
#               8: b1_lin, 9: b2_lin, 10:b3_lin, 11:b4_lin,
#               12:c1_lin, 13:c2_lin, 14:c3_lin, 15:c4_lin,
#               16:x0_1_lin, 17:x0_2_lin, 18:x0_3_lin, 19:x0_4_lin,
#               20:y0_1_lin, 21:y0_2_lin, 22:y0_3_lin, 23:y0_4_lin}

# param_dict = {0: "A1", 1: "A2", 2: "A3", 3: "A4",
#               4: "a1", 5: "a2", 6: "a3", 7: "a4",
#               8: "b1", 9: "b2", 10:"b3", 11:"b4",
#               12:"c1", 13:"c2", 14:"c3", 15:"c4",
#               16:"x0_1", 17:"x0_2", 18:"x0_3", 19:"x0_4"}

# param_dict_vals = {0: A1_lin, 1: A2_lin, 2: A3_lin, 3: A4_lin,
#               4: a1_lin, 5: a2_lin, 6: a3_lin, 7: a4_lin,
#               8: b1_lin, 9: b2_lin, 10:b3_lin, 11:b4_lin,
#               12:c1_lin, 13:c2_lin, 14:c3_lin, 15:c4_lin,
#               16:x0_1_lin, 17:x0_2_lin, 18:x0_3_lin, 19:x0_4_lin}

param_dict = {0: "A1", 1: "A2", 2: "A3", 3: "A4",
              4: "a1", 5: "a2", 6: "a3", 7: "a4",
              8: "b1", 9: "b2", 10:"b3", 11:"b4",
              12:"c1", 13:"c2", 14:"c3", 15:"c4",
              16:"x0_1", 17:"x0_2", 18:"x0_3", 19:"x0_4"}

param_dict_vals = {0: A1_lin, 1: A2_lin, 2: A3_lin, 3: A4_lin,
              4: a1_lin, 5: a2_lin, 6: a3_lin, 7: a4_lin,
              8: b1_lin, 9: b2_lin, 10:b3_lin, 11:b4_lin,
              12:c1_lin, 13:c2_lin, 14:c3_lin, 15:c4_lin,
              16:x0_1_lin, 17:x0_2_lin, 18:x0_3_lin, 19:x0_4_lin}

skip_params = 1
dim_list = np.linspace(0,d-1,d)
mesh_combos = np.array(list(combinations(dim_list, 2)), dtype = int)
# print(mesh_combos)
#Loop over combinations of axes and create heatmaps
for i in range(len(mesh_combos)):
    indecies = mesh_combos[i] +4*skip_params
#     print( [ param_dict[indecies[0]], param_dict[indecies[1]] ])
    param_vals_list = [ param_dict_vals[indecies[0]], param_dict_vals[indecies[1]] ]
    P1_vals, P2_vals = param_vals_list[0], param_vals_list[1]
    theta_mesh = np.array(np.meshgrid(P1_vals, P2_vals))
#     print(theta_mesh)
    zz = sse_func(a_model, Xexp, Yexp, theta_mesh, indecies, skip_params)
#     print(zz)

    if log_plot == True:
        zz = np.log(zz)
    #Better way to do this?
    plt.figure(figsize = (6,6))

    plt.contourf(P1_vals,P2_vals, zz, cmap = "autumn", levels = 100)
#             plt.colorbar()

    cs = plt.contourf(P1_vals,P2_vals, zz, cmap = "autumn", levels = 100)
            # plot color bar
    if np.amax(zz) < 1e-1:
        cbar = plt.colorbar(cs, format='%.2e')
    else:
        cbar = plt.colorbar(cs, format = '%2.2f')

        # plot title in color bar
        cbar.ax.set_ylabel(r'$\mathbf{log(e(\theta))}$', fontsize=16, fontweight='bold')
    #     print(p_GP_opt[0],p_GP_opt[1])

        # set font size in color bar
        cbar.ax.tick_params(labelsize=16)

        # Plot equipotential line
        cs2 = plt.contour(cs, levels=cs.levels[::20], colors='k', alpha=0.7, linestyles='dashed', linewidths=3)

        #Plot heatmap label

        if np.amax(zz) < 1e-1:
            plt.clabel(cs2, fmt='%.2e', colors='k', fontsize=16)
        else:
            plt.clabel(cs2, fmt='%2.2f', colors='k', fontsize=16)

        plt.axis()
        
#         print(Constants_2[indecies[0]+4*skip_params])
        plt.scatter(Constants_2[indecies[0]],Constants_2[indecies[1]], color="blue", s=100, 
                    label = "True Optimal Value", marker = (5,1)) #k +1 since we chose a&b
        plt.scatter(a_model[indecies[0]- 4*skip_params],a_model[indecies[1]- 4*skip_params], color="white",s=50, 
                    marker = ".", edgecolors= "k", linewidth=0.3, label = "NLR Optimal Value")
#         print(a_model[indecies[0]+4*skip_params], Constants_2[indecies[0]+4*skip_params])
#         print(a_model[indecies[1]+4*skip_params], Constants_2[indecies[1]+4*skip_params])
        # plt.grid()
        plt.legend(fontsize=10,bbox_to_anchor=(0, 1.05, 1, 0.2),borderaxespad=0, loc = "lower right")
        plt.xlabel(param_dict[indecies[0]],fontsize=16,fontweight='bold')
        plt.ylabel(param_dict[indecies[1]],fontsize=16,fontweight='bold')

        plt.xlim((np.amin(P1_vals), np.amax(P1_vals)))
        plt.ylim((np.amin(P2_vals), np.amax(P2_vals)))

        plt.minorticks_on() # turn on minor ticks
        plt.xticks(fontsize=16)
        plt.yticks(fontsize=16)
        plt.tick_params(which="minor",direction="in",top=True, right=True)
        plt.locator_params(axis='y', nbins=5)
        plt.locator_params(axis='x', nbins=5)

        if save_figure == True:
#             plt.title('NLR ln(SSE)', weight='bold',fontsize = 16)
            path = "2023/NLR/CS_" + str(CS) +"/log_sse/" + param_dict[indecies[0]] + "-" + param_dict[indecies[1]]
            save_fig(path, ext='png', close=True, verbose=False)
        else:
            plt.show()

In [ ]:
#Jacobian and Uncertainty Analysis
print("Jacobian =\n")
Jacobian = Solution.jac
print(Jacobian) #Jacobian is fine for simple cases
#Normalize Jacobian
# print("Normalized Jacobian =\n")
# for i in range(len(a_model)):
#     Jacobian[i,:] = Jacobian[i,:]*a_model
# print(Jacobian)

#OR normalize error instead of FIM to solve the problem (2 ways to do this)
# error_normalized = (error - np.average(error)) / np.average(error)
# error_normalized = (error - np.amin(error)) / (np.amax(error)- np.amin(error))
# print("Normalized Error =  \n", error_normalized)

sigre = (error.T @ error)/(len(error) - 2)
# sigre = (error_normalized.T @ error_normalized)/(len(error_normalized) - 2)
# Sigma_theta2 = sigre * np.linalg.inv(Solution.jac.T @ Solution.jac)
Sigma_theta2 = sigre * np.linalg.inv(Jacobian.T @ Jacobian)
print("Covariance matrix:\n",Sigma_theta2)

In [ ]:
import scipy.linalg as linalg
val, vec = linalg.eig(Sigma_theta2)
print("Eigenvalues = \n",val)
print("Eigenvectors = \n",vec)

In [ ]:
FIM = (1/sigre) * Jacobian.T @ Jacobian
print("FIM = \n", FIM)

In [ ]:
print(np.linalg.det(FIM))
print(np.linalg.det(Sigma_theta2))
print(sigre)

In [ ]:
Test = Jacobian.T @ Jacobian
print(np.linalg.det(Test))
print(Test)
val, vec = linalg.eig(Test)
print(val)